In [8]:
import motmetrics as mm
from pathlib import Path
import os
import pandas as pd
import json
import numpy as np

In [9]:
FOLDER_NAME = str(input("Enter your folder name (e.g., '20251229-223351')"))

DATA_DIR = os.path.join(Path.cwd(), 'data')
PREDICTED_DIR = os.path.join(DATA_DIR, 'predicted', FOLDER_NAME)
CONFIG_DIR = os.path.join(DATA_DIR, 'config', FOLDER_NAME)
GROUND_TRUTH_FILE = os.path.join(DATA_DIR, 'ground-truth', 'test_30s_output.txt')
EVALUATE_DIR = os.path.join(DATA_DIR, 'evaluate')

os.makedirs(EVALUATE_DIR, exist_ok=True)

if not os.path.exists(PREDICTED_DIR) or not os.path.isdir(PREDICTED_DIR) or len(os.listdir(PREDICTED_DIR)) == 0:
    raise FileNotFoundError(f"The directory {PREDICTED_DIR} does not exist or is empty.")
if not os.path.exists(GROUND_TRUTH_FILE):
    raise FileNotFoundError(f"The ground truth file {GROUND_TRUTH_FILE} does not exist.")
if not os.path.exists(CONFIG_DIR) or not os.path.isdir(CONFIG_DIR) or len(os.listdir(CONFIG_DIR)) == 0:
    raise FileNotFoundError(f"The directory {CONFIG_DIR} does not exist or is empty.")
print(f"All necessary files and directories are present.")

All necessary files and directories are present.


In [10]:
df = pd.DataFrame()

In [11]:
def evaluate_tracker(gt_file, ts_file, name):
    gt = mm.io.loadtxt(gt_file, fmt='mot16', min_confidence=1)
    ts = mm.io.loadtxt(ts_file, fmt='mot16')

    acc = mm.utils.compare_to_groundtruth(gt, ts, 'iou', distth=0.5)

    mh = mm.metrics.create()
    summary = mh.compute(acc, metrics=mm.metrics.motchallenge_metrics, name=name)

    row = summary.loc[name].to_dict()
    return row

def evaluate_config(config_file):
    with open(config_file, 'r') as f:
        data = json.load(f)

    algorithm = data['algorithm']
    attempt = data['attempt']
    inference_time = data['inference_time']
    tracking_time = data['tracking_time']

    return algorithm, attempt, inference_time, tracking_time

def add_metadata(row, algorithm, attempt, inference_time, tracking_time, std_inference_time, std_tracking_time):
    row.update({
        'algorithm': algorithm,
        'attempt': attempt,
        'inference_time': inference_time,
        'tracking_time': tracking_time,
        'std_inference_time': std_inference_time,
        'std_tracking_time': std_tracking_time
    })
    return row

def store_results(row):
    global df
    df = pd.concat([df, pd.DataFrame([row])], ignore_index=True)

In [12]:
for config_file in os.listdir(CONFIG_DIR):
    print(config_file)
    file_name = config_file.split('.')[0]

    algorithm, attempt, inference_time, tracking_time = evaluate_config(os.path.join(CONFIG_DIR, config_file))
    avg_inference_time = np.mean(inference_time)
    avg_tracking_time = np.mean(tracking_time)
    std_inference_time = np.std(inference_time)
    std_tracking_time = np.std(tracking_time)

    predict_file = os.path.join(PREDICTED_DIR, f"{file_name}.txt")
    if not os.path.exists(predict_file):
        print(f"{config_file} cann't locate the detection csv")
        break

    row = evaluate_tracker(GROUND_TRUTH_FILE, predict_file, file_name)
    row = add_metadata(row, algorithm, attempt, avg_inference_time, avg_tracking_time, std_inference_time, std_tracking_time)
    store_results(row)

BoostTrack_round1.json
BoostTrack_round10.json
BoostTrack_round2.json
BoostTrack_round3.json
BoostTrack_round4.json
BoostTrack_round5.json
BoostTrack_round6.json
BoostTrack_round7.json
BoostTrack_round8.json
BoostTrack_round9.json
BotSort_round1.json
BotSort_round10.json
BotSort_round2.json
BotSort_round3.json
BotSort_round4.json
BotSort_round5.json
BotSort_round6.json
BotSort_round7.json
BotSort_round8.json
BotSort_round9.json
ByteTrack_round1.json
ByteTrack_round10.json
ByteTrack_round2.json
ByteTrack_round3.json
ByteTrack_round4.json
ByteTrack_round5.json
ByteTrack_round6.json
ByteTrack_round7.json
ByteTrack_round8.json
ByteTrack_round9.json
DeepOcSort_round1.json
DeepOcSort_round10.json
DeepOcSort_round2.json
DeepOcSort_round3.json
DeepOcSort_round4.json
DeepOcSort_round5.json
DeepOcSort_round6.json
DeepOcSort_round7.json
DeepOcSort_round8.json
DeepOcSort_round9.json
HybridSort_round1.json
HybridSort_round10.json
HybridSort_round2.json
HybridSort_round3.json
HybridSort_round4.json


In [13]:
display(df)

,idf1,idp,idr,recall,precision,num_unique_objects,mostly_tracked,partially_tracked,mostly_lost,num_false_positives,...,motp,num_transfer,num_ascend,num_migrate,algorithm,attempt,inference_time,tracking_time,std_inference_time,std_tracking_time
0,0.103710,0.118082,0.092323,0.265277,0.339923,42.0,4.0,9.0,29.0,4274.0,...,0.400778,424.0,5.0,24.0,BoostTrack,1,0.005856,0.005221,0.002124,0.003361
1,0.103710,0.118082,0.092323,0.265277,0.339923,42.0,4.0,9.0,29.0,4274.0,...,0.400778,424.0,5.0,24.0,BoostTrack,10,0.011192,0.005829,0.001690,0.003400
2,0.103710,0.118082,0.092323,0.265277,0.339923,42.0,4.0,9.0,29.0,4274.0,...,0.400778,424.0,5.0,24.0,BoostTrack,2,0.005364,0.005306,0.001886,0.003719
3,0.103710,0.118082,0.092323,0.265277,0.339923,42.0,4.0,9.0,29.0,4274.0,...,0.400778,424.0,5.0,24.0,BoostTrack,3,0.004581,0.005291,0.002038,0.003501
4,0.103710,0.118082,0.092323,0.265277,0.339923,42.0,4.0,9.0,29.0,4274.0,...,0.400778,424.0,5.0,24.0,BoostTrack,4,0.005218,0.005107,0.001706,0.003317
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,0.107074,0.098816,0.116669,0.348560,0.295585,42.0,11.0,9.0,22.0,6892.0,...,0.390782,674.0,7.0,24.0,StrongSort,5,0.005371,0.044021,0.000756,0.021533
66,0.107074,0.098816,0.116669,0.348560,0.295585,42.0,11.0,9.0,22.0,6892.0,...,0.390790,674.0,7.0,24.0,StrongSort,6,0.005231,0.043357,0.001129,0.023485
67,0.107074,0.098816,0.116669,0.348560,0.295585,42.0,11.0,9.0,22.0,6892.0,...,0.390791,674.0,7.0,24.0,StrongSort,7,0.005152,0.044157,0.001180,0.022112
68,0.107074,0.098816,0.116669,0.348560,0.295585,42.0,11.0,9.0,22.0,6892.0,...,0.390792,674.0,7.0,24.0,StrongSort,8,0.005568,0.045761,0.000872,0.024492


In [14]:
want_to_save = str(input("Do you want to save the results to a CSV file? (yes/no): ")).strip().lower()
if want_to_save == 'yes':
    output_file = os.path.join(EVALUATE_DIR, f"evaluation_results_{FOLDER_NAME}.csv")
    df.to_csv(output_file, index=False)
    print(f"Results saved to {output_file}")
else:
    print("Results not saved.")

Results saved to c:\Users\sschw\schwynn\Work\Teacher-Supervised\traffic\benchmark\data\evaluate\evaluation_results_20251229-234204.csv
